In [ ]:
import pandas as pd
import requests as r
from bs4 import BeautifulSoup

fighters = pd.read_csv('fighter_sot.csv', encoding='iso-8859-1', low_memory=False)

In [ ]:
not_missing = fighters['type'] == 'e'
not_found = fighters['fighter_name'].isnull()

In [ ]:
only_errors = fighters[not_found & not_missing]

In [ ]:
correct = [1,2,3]
incorrect = [128663, 128669, 128769]
sample = correct + incorrect

In [ ]:
def parse_fighter_page(page_soup):
    fighter_info = {'figher_name':'no_fighter', 'type':'m'}
    matches = None
    try:
        #get fighter name
        # set fighter info
    except:
        pass
    try:
        # get matches with strategy 1
        # matches = blah
    except:
        pass
    try:
        # get matches with strategy 2
        # matches = blah2
    except: # if wrong strategy error thrown
        pass
    return fighter_info, matchesc
    


def verify(row):
    name = row['fighter_name']
    fighter_id = row['id']
    row_type = row['type']
    #fighter_page = soup(lookup fighter id)
    # fighter_info, matches = parser(fighter_page)
    # row['fighter_name'] = fighter_info['fighter_name']
    # row['type'] = fighter_info['type']
    # add matches to match df
    return row
    
only_errors = only_errors.apply(verify, axis=1)